In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
import httpagentparser

In [10]:
from clickhouse_driver import Client


client = Client('localhost', settings={'use_numpy': True})
def query_df(query):
    "Execute query and save it to pandas.DataFrame"
    result, columns = client.execute(query, with_column_types=True)
    return pd.DataFrame(result, columns=[v[0] for v in columns])

In [16]:
# посмотрим на данные в таблице об устройствах
# для решения задания принял, что hostname - устройство, а client - пользователь
query_df("""select * from test_db.clients_info limit 10""")

,client,hostname,alias_list,addres_list
0,1.132.107.223,1.132.107.223,[Errno 1] Unknown host,None
1,1.132.108.133,1.132.108.133,[Errno 1] Unknown host,None
2,1.136.111.52,1.136.111.52,[Errno 1] Unknown host,None
3,1.158.74.178,1.158.74.178,[Errno 1] Unknown host,None
4,1.159.185.202,cpe-1-159-185-202.wb05.wa.asp.telstra.net,['202.185.159.1.in-addr.arpa'],['1.159.185.202']
5,1.214.221.2,1.214.221.2,[Errno 1] Unknown host,None
6,1.234.99.77,1.234.99.77,[Errno 1] Unknown host,None
7,1.36.128.224,1.36.128.224,[Errno 1] Unknown host,None
8,1.43.175.52,1.43.175.52,[Errno 1] Unknown host,None
9,1.46.105.255,1.46.105.255,[Errno 1] Unknown host,None


In [28]:
# создаем таблицу для данных из логов
client.execute("""
create table test_db.log_info(
IP String,
timestamp String,
response Int16,
numbers Int32,
device_name String,
browser String
)
ENGINE = MergeTree()
ORDER BY (IP, timestamp);
""")

[]

In [48]:
# собираем информацию из логов
import re
result_df = pd.DataFrame()
d_dict = {}
d_dict['IP'] = []
d_dict['timestamp'] = []
d_dict['response'] = []
d_dict['numbers'] = []
d_dict['device_name'] = []
d_dict['browser'] = []
regex = '\"(.*?)\"|\[(.*?)\]|(\S+)'
with open('access.log','r') as file:
    for line in file:
#         print(line)
        parsed_line = re.findall(r'\"(.*?)\"|\[(.*?)\]|(\S+)', line)
        d_dict['IP'].append(parsed_line[0][2])
        d_dict['timestamp'].append(parsed_line[3][1])
        d_dict['response'].append(parsed_line[5][2])
        d_dict['numbers'].append(parsed_line[6][2])
        try:
            d_dict['device_name'].append(parsed_line[-2][0].split("(")[1].split(")")[0].split(";")[-1])
        except(BaseException):
            d_dict['device_name'].append(None)
        try:
            d_dict['browser'].append(httpagentparser.detect(line)['browser']['name'])
        except(BaseException):
            d_dict['browser'].append(None)
#         print(line)
#         print(re.findall(r'\"(.*?)\"|\[(.*?)\]|(\S+)', line))
#         print(' ')
#         print("IP-adress - ", re.findall(r'\"(.*?)\"|\[(.*?)\]|(\S+)', line)[0][2])
#         print('date_time - ', re.findall(r'\"(.*?)\"|\[(.*?)\]|(\S+)', line)[3][1])
#         print('response - ', re.findall(r'\"(.*?)\"|\[(.*?)\]|(\S+)', line)[5][2])
#         print('surr_key - ', re.findall(r'\"(.*?)\"|\[(.*?)\]|(\S+)', line)[6][2])
#         print('last - ', re.findall(r'\"(.*?)\"|\[(.*?)\]|(\S+)', line)[-2][0])
#         print('device_name - ', re.findall(r'\"(.*?)\"|\[(.*?)\]|(\S+)', line)[-2][0].split("(")[1].split(")")[0].split(";")[-1])
#         print('device info - ', httpagentparser.detect(re.findall(r'\"(.*?)\"|\[(.*?)\]|(\S+)', line)[-2][0]))
#         print(' ')

data = pd.DataFrame.from_dict(d_dict)
data.to_csv('parsed_logs_2.csv')

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10365152 entries, 0 to 10365151
Data columns (total 6 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   IP           object
 1   timestamp    object
 2   response     int32 
 3   numbers      int64 
 4   device_name  object
 5   browser      object
dtypes: int32(1), int64(1), object(4)
memory usage: 514.0+ MB


In [29]:
# закидываем данные в базу
client.insert_dataframe("""insert into test_db.log_info values""", data)

10365152

In [30]:
data

,IP,timestamp,response,numbers,device_name,browser
0,54.36.149.41,22/Jan/2019:03:56:14 +0330,200,30577,+http://ahrefs.com/robot/,NaN
1,31.56.96.51,22/Jan/2019:03:56:16 +0330,200,5667,ALE-L21 Build/HuaweiALE-L21,Chrome
2,31.56.96.51,22/Jan/2019:03:56:16 +0330,200,5379,ALE-L21 Build/HuaweiALE-L21,Chrome
3,40.77.167.129,22/Jan/2019:03:56:17 +0330,200,1696,+http://www.bing.com/bingbot.htm,BingBot
4,91.99.72.15,22/Jan/2019:03:56:17 +0330,200,41483,rv:16.0,Firefox
...,...,...,...,...,...,...
10365147,188.229.21.56,26/Jan/2019:20:29:13 +0330,302,0,SM-N920C Build/NRD90M,Chrome
10365148,5.127.220.71,26/Jan/2019:20:29:13 +0330,404,32420,NaN,Safari
10365149,5.213.7.50,26/Jan/2019:20:29:13 +0330,200,20959,CPU iPhone OS 10_2_1 like Mac OS X,Safari
10365150,109.125.169.52,26/Jan/2019:20:29:13 +0330,200,5,rv:64.0,Firefox


In [ ]:
# соединим таблицы вместе во вьюхе
client.execute("""
create view test_db.twin_table (
IP String,
timestamp String,
response Int16,
numbers Int32,
device_name String,
browser String,
client String,
hostname String,
alias_list String,
addres_list Nullable(String))
ENGINE = MergeTree()
order by (IP, timestamp)
as select * 
from log_info li 
join clients_info ci
on li.IP = ci.client 
""")

In [40]:
# Создадим первую витрину с 7-ю пунктами из задания
#1. Суррогатный ключ устройства
#2. Название устройства
#3. Количество пользователей
#4. Доля пользователей данного устройства от общего числа пользователей
#5. Количество совершенных действий для данного устройства
#6. Доля совершенных действий с данного устройства относительно других устройств
#8. Количество ответов сервера, отличных от 200 на данном устройстве 
client.execute(
"""create view test_db.first_view as select * from (
with sur_key as (select 
generateUUIDv4() as surr_key, -- Суррогатный ключ устройства
hostname, -- Название устройства
count(hostname) as cnt_actions, --Количество совершенных действий для данного устройства
((count(hostname)/(select count(*) from test_db.log_info))*100) as percent_from_all -- Доля совершенных действий с данного устройства относительно других устройств
from test_db.twin_table tt 
group by hostname),
cli_info as (select 
hostname, -- Название устройства
count(client) as users, -- Количество пользователей
((count(client)/ (select count(*) from test_db.clients_info))*100) as user_percent_by_host -- Доля пользователей данного устройства от общего числа пользователей
from test_db.clients_info ci  
group by hostname),
response as (select hostname, 
count() as not_200_sum -- Количество ответов сервера, отличных от 200 на данном устройстве
from test_db.twin_table tt 
where response != 200
group by hostname
order by hostname )
select 
sur_key.surr_key,
sur_key.hostname,
cli_info.user_percent_by_host,
sur_key.cnt_actions,
sur_key.percent_from_all,
response.not_200_sum
from sur_key
join cli_info
on sur_key.hostname = cli_info.hostname
join response
on sur_key.hostname = response.hostname)
""")

[]

In [36]:
#  не знаю как ограничить до 5 самых популярных браузеров, поэтому пока выводятся все, но это не так критично, 
#  потому как максимум 7 браузеров и всего на нескольких устройствах (до 10) больше 5 браузеров

#7. Список из 5 самых популярных браузеров, используемых на данном устройстве различными пользователями, 
#   с указанием доли использования для данного браузера относительно остальных браузеров 
client.execute(
"""create view test_db.second_view as
SELECT hostname, browser, 
count(*) as brows_actions, 
count(hostname) over (partition by hostname) as cnt_brws_on_ip,
((count(*)/(sum(count(*)) over (partition by hostname)))*100) as percent_by_browser_action
from test_db.twin_table tt 
group by hostname, browser 
--order by hostname
""")

[]

In [37]:
# таблица небольщая, поэтому не стал исключать информацию по ответу 200, но можно его убрать, убрав закомментированную часть кода
# 9. Для каждого из ответов сервера, отличных от 200, сформировать поле, в котором будет содержаться количество ответов данного типа
client.execute(
"""create view test_db.third_view as
select 
response, 
count() total__by_response
from test_db.twin_table tt 
--where response != 200
group by response""")


[]

In [44]:
# Посмотрим на данные из витрины 1
query_df("""select * from test_db.first_view limit 10""")

NumPy support is not implemented for UUID. Using generic column


,sur_key.surr_key,sur_key.hostname,cli_info.user_percent_by_host,sur_key.cnt_actions,sur_key.percent_from_all,response.not_200_sum
0,9b059493-f54d-4c4b-bb35-06b44a0a5265,5.117.118.154,0.000387,214,0.002065,3
1,fa54b965-b042-4346-b511-26b2d8e7b513,5.123.72.214,0.000387,32,0.000309,3
2,d4fc4d7d-04da-4757-9dbe-70d529d5e0e8,5.115.163.193,0.000387,22,0.000212,16
3,8b0a7d99-6777-4549-bb64-3cec8ba937e5,91.106.86.212,0.000387,133,0.001283,18
4,2da72534-4f15-4923-939c-b5918c906753,5.115.166.29,0.000387,29,0.000280,2
5,91e28d44-68b0-4e94-ae9e-bb215f268774,151.239.13.34,0.000387,37,0.000357,1
6,fa50640d-564c-46de-ab1d-5ab43a33bec5,2.177.149.74,0.000387,24,0.000232,1
7,b3bf26a0-86c7-4386-9997-eda22d434ae7,91.251.163.67,0.000387,11,0.000106,8
8,dddae628-1ae8-42c1-aec5-04481ec43384,194.60.238.85,0.000387,1,0.000010,1
9,cc31d319-b9c2-4365-8d96-6f42b72800f0,5.120.190.2,0.000387,40,0.000386,2


In [45]:
# посмотрим на данные из витрины 2
query_df("""select * from test_db.second_view limit 10""")

,hostname,browser,brows_actions,cnt_brws_on_ip,percent_by_browser_action
0,,Chrome,589,4,99.32546374367622
1,,Firefox,1,4,0.16863406408094433
2,,Opera,1,4,0.16863406408094433
3,,AndroidBrowser,2,4,0.33726812816188867
4,014199038003.ctinets.com,Microsoft Internet Explorer,1,1,100.0
5,027aa2c9.bb.sky.com,Safari,12,1,100.0
6,0540bdab.skybroadband.com,Chrome,36,1,100.0
7,071-080-018-092.res.spectrum.com,Chrome,1,1,100.0
8,096-041-104-003.res.spectrum.com,,1,1,100.0
9,1.132.107.223,Chrome,15,1,100.0


In [47]:
#  Посмотрим на данные из витрины 3
query_df("""select * from test_db.third_view""")

,response,total__by_response
0,200.0,9579825.0
1,206.0,3.0
2,301.0,67552.0
3,302.0,199835.0
4,304.0,340228.0
5,400.0,358.0
6,401.0,323.0
7,403.0,5634.0
8,404.0,105011.0
9,405.0,6.0
